## Graticule Code Test

In [1]:
import conjugate_map.conj_calc as conj
import datetime as dt
import pandas as pd

import logging
logger = logging.getLogger(__name__)

Load IGRF coefficients ...


In [2]:
import sys
sys.version

'3.9.1 (default, Sep 16 2022, 17:01:50) \n[GCC 7.5.0]'

In [3]:
from geopack import geopack as gp


In [4]:
help(gp)

Help on module geopack.geopack in geopack:

NAME
    geopack.geopack

FUNCTIONS
    bcarsp(x, y, z, bx, by, bz)
        Calculates spherical field components from those in cartesian system
        
        :param x,y,z: cartesian components of the position vector
        :param bx,by,bz: cartesian components of the field vector
        :return: br,btheta,bphi. spherical components of the field vector
        
        Last mofification:  April 1, 2003 (only some notation changes and more comments added)
        Author:  N.A. Tsyganenko
    
    bspcar(theta, phi, br, btheta, bphi)
        Calculates cartesian field components from spherical ones.
        
        :param theta,phi: spherical angles of the point in radians
        :param br,btheta,bphi: spherical components of the field
        :return: bx,by,bz. cartesian components of the field
        
        # Last mofification:  April 1, 2003 (only some notation changes and more comments added)
        # Author:  N.A. Tsyganenko
   

### Compute conjugate points for one point:

In [5]:
ut = dt.datetime(1980, 11, 3, 18, 0, 0)
lat, lon = [-88, -0]

print("Geopack: ")
print(conj.findconj(lat, lon, ut, method="geopack"))
print("AACGM: ")
print(conj.findconj(lat, lon, ut, method="aacgm"))
print("Quasi-dipole: ")
print(conj.findconj(lat, lon, ut, method="qdip"))
print("\nAutomatic: ")
print(conj.findconj(lat, lon, ut, method="auto"))


Geopack: 
(64.49431381411007, 298.24582917033456)
AACGM: 
(63.10539805333622, -62.838832111320556)
Quasi-dipole: 
(63.04719543457031, -62.91141891479492)

Automatic: 
(63.10539805333622, -62.838832111320556)


## Compute conjugate points for all points in a dataframe:

First we declare a dataframe:

In [6]:
df = pd.DataFrame({'Name':['McMurdo', 'SPA', 'PGC', 'SSI'], 
                   'lat': [-77.846323, -90, 44.984308, 40.019511], 
                   'lon': [ 166.668235, 0, -93.182207, -105.240014]
                  })

df

,Name,lat,lon
0,McMurdo,-77.846323,166.668235
1,SPA,-90.000000,0.000000
2,PGC,44.984308,-93.182207
3,SSI,40.019511,-105.240014


...and then we compute the conjugate points:

In [7]:
conj.conjcalc(df, dtime = ut, latname = 'lat', lonname = 'lon')

/home/pliny/.local/lib/python3.9/site-packages/conjugate_map/conj_calc.py:278: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'S' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  gdf.loc[index, 'Hemisphere'] = "S"


,Name,lat,lon,Hemisphere,PLAT,PLON
0,McMurdo,-77.846323,166.668235,S,69.678409,-92.264039
1,SPA,-90.000000,0.000000,S,63.904820,-66.297775
2,PGC,44.984308,-93.182207,N,44.984308,-93.182207
3,SSI,40.019511,-105.240014,N,40.019511,-105.240014


The default values are documented in the function:

In [8]:
help(conj.conjcalc)

Help on function conjcalc in module conjugate_map.conj_calc:

conjcalc(gdf, latname='GLAT', lonname='GLON', dtime=datetime.datetime(2025, 6, 10, 22, 37, 39, 809635, tzinfo=datetime.timezone.utc), method='aacgm', mode='S2N', is_saved=False, directory='output/', name='stations')
    Calculate the geographic latitudes and longitudes of conjugate points
    for all points in a dataframe. Calls findconj().
    
    Parameters
    ----------
    gdf         : dataframe of points whose conjugate points we're finding
    lat         : float
            Geographic latitude of station.
    lon         : float
            Geographic longitude of station.
    ut          : datetime
            Datetime used in conversion.
    method      : string
            Defines method used in conversion. Options are 'auto', 'geopack',
            which uses IGRF + T89 to run field line traces,
            or 'aacgm', which uses AACGM v2.
    limit       : float
            Latitude limit, in degrees, used to 

## Working on graticule calcs...

In [9]:
# inputs
import datetime
mlats = [-80, 65, 65, 80]
ut = datetime.datetime(2007, 1, 1)
method = "auto"
is_saved=False

In [10]:
import numpy as np
import aacgmv2
import gpxpy
import gpxpy.gpx as gpx

In [11]:
help(aacgmv2.convert_latlon)

Help on function convert_latlon in module aacgmv2.wrapper:

convert_latlon(in_lat, in_lon, height, dtime, method_code='G2A')
    Convert between geomagnetic coordinates and AACGM coordinates.
    
    Parameters
    ----------
    in_lat : float
        Input latitude in degrees N (code specifies type of latitude)
    in_lon : float
        Input longitude in degrees E (code specifies type of longitude)
    height : float
        Altitude above the surface of the earth in km
    dtime : dt.datetime
        Datetime for magnetic field
    method_code : str or int
        Bit code or string denoting which type(s) of conversion to perform
        (default="G2A")
    
        G2A
            Geographic (geodetic) to AACGM-v2
        A2G
            AACGM-v2 to geographic (geodetic)
        TRACE
            Use field-line tracing, not coefficients
        ALLOWTRACE
            Use trace only above 2000 km
        BADIDEA
            Use coefficients above 2000 km
        GEOCENTRIC
      

In [12]:
# mlons = np.arange(0, 360)
# mlats_dct = {}
# for mlat in mlats:
#     glats = []
#     glons = []
#     for mlon in mlons:
        
#         if method == "aacgm":
#             result = aacgmv2.convert_latlon(mlat, mlon, 0, ut, 'A2G')
#         else:
#             # todo - calculate 
#             # make glats and glons into a dataframe
#             # activate conjcalc with that dataframe
#             result = aacgmv2.convert_latlon(mlat, mlon, 0, ut, 'A2G')
#         glats.append(result[0])
#         glons.append(result[1])

#     mlats_dct[mlat] = {'glats': glats, 'glons': glons}

#     if is_saved is True:
#         logger.info('Saving magnetic graticule for %f degrees magnetic latitude.', mlat)  # noqa: E501
#         filename = 'Graticule_ ' + str(mlat) + '_' + str(ut) + '_' + method
#         directory = 'output/'
#         df = pd.DataFrame({'MLAT': mlats_dct[mlat]['glats'],
#                            'MLON': mlats_dct[mlat]['glons']})

#         f = os.path.join(directory, filename)
#         gpx = gpxpy.gpx.GPX()

#         # Create first track in our GPX:
#         gpx_track = gpxpy.gpx.GPXTrack()
#         gpx.tracks.append(gpx_track)

#         # Create first segment in our GPX track:
#         gpx_segment = gpxpy.gpx.GPXTrackSegment()
#         gpx_track.segments.append(gpx_segment)

#         # Create points:
#         for idx in df.index:
#             gpx_segment.points.append(gpxpy.gpx.GPXTrackPoint(
#                 df.loc[idx, 'MLAT'], df.loc[idx, 'MLON']))

#         logger.info(gpx.to_xml())

#         with open('output/graticules/'+filename+'.gpx', 'w',
#                   encoding="utf-8") as f:
#             f.write(gpx.to_xml())
#             logger.info("Writing %s to gpx. ", filename)
# # return mlats_dct
# pd.DataFrame(mlats_dct)

In [13]:
# import datetime as dt
# import os
# import numpy as np
# import pandas as pd
# import gpxpy
# import gpxpy.gpx
# import json
# import aacgmv2
# import apexpy as apex # Assuming apexpy is imported as apex
# import geopack as gp # Assuming geopack is imported as gp

# # Assuming 'logger' is defined elsewhere in your script, e.g.:
# import logging
# logger = logging.getLogger(__name__)
# logging.basicConfig(level=logging.INFO)

# def calc_mlat_rings(mlats, ut=dt.datetime.now(tz=dt.timezone.utc),
#                     method="auto",
#                     is_saved=False):
#     """Calculate the geographic latitudes and longitudes of a circle of points
#     for a list of magnetic latitudes. Provide a latitude in the magnetic coordinate
#     system of your choice, and this function will return a set of geographic
#     coordinates for points along that magnetic latitude.

#     Parameters
#     ----------
#     mlats       : np.array
#             List of magnetic latitudes
#     ut          : dt.datetime
#             Datetime used in AACGMv2 conversion;
#             by default, ut=dt.datetime.now(tz=dt.timezone.utc)
#     method      : string
#             Defines coordinate system used.
#             "aacgm": Calculated using aacgmv2.
#             "qdip" : Calculated using apexpy.
#             "apex" : Calculated using apexpy.
#             "gsm"  : Calculated using geopack.
#             "auto" : defaults to aacgm.

#     is_saved    : boolean
#             If is_saved == True, saves .gpx and .geojson versions
#             to local output directory.


#     Returns
#     -------
#     mlats_dct: dict
#         Dictionary with geographic latitude and longitude
#         points for each of the specified magnetic latitudes.

#     Example Use
#     ------------
#     Saves .gpx magnetic graticules for 1 January 2020 every 5
#     degrees latitude::

#         rings = calc_mlat_rings(list(range(-90, 90, 5)), ut =
#                                 dt.datetime(2020, 1, 1), is_saved = True)

#     """
#     mlons = np.arange(0, 360)
#     mlats_dct = {}
#     for mlat in mlats:
#         glats = []
#         glons = []
#         for mlon in mlons:
#             if method == "aacgm" or method == "auto": # Fixed the 'or "auto"' logic
#                 result = aacgmv2.convert_latlon(mlat, mlon, 0, ut, 'A2G')
#             elif method == "qdip":
#                 # Assuming height=0 km for ground level
#                 glat, glon = apex.qd2geo(mlat, mlon, 0)
#                 result = (glat, glon, 0)  # Format similar to aacgmv2 result
#             elif method == "apex":
#                 # Assuming height=0 km for ground level
#                 glat, glon = apex.apex2geo(mlat, mlon, 0)
#                 result = (glat, glon, 0)  # Format similar to aacgmv2 result
#             elif method == "gsm":
#                 # magnetic lat/lon to XYZ:
#                 R_E = 6371.0  # km
#                 x_gsm = R_E * np.cos(np.deg2rad(mlat)) * np.cos(np.deg2rad(mlon))
#                 y_gsm = R_E * np.cos(np.deg2rad(mlat)) * np.sin(np.deg2rad(mlon))
#                 z_gsm = R_E * np.sin(np.deg2rad(mlat))

#                 # Get dipole tilt angle for geopack
#                 ps = gp.recalc(ut.year, ut.month, ut.day, ut.hour, ut.minute, ut.second)

#                 # magnetic XYZ (GSM) to geographic XYZ (GEO)
#                 # Note: gp.geogsm converts GEO to GSM, so we need the inverse, gp.gsmgeo.
#                 x_geo, y_geo, z_geo = gp.gsmgeo(x_gsm, y_gsm, z_gsm, ps)

#                 # geographic XYZ to geographic lat/lon
#                 # Convert Cartesian GEO (x_geo, y_geo, z_geo) to Spherical GEO (glat, glon, radius)
#                 # Handle potential division by zero if x_geo, y_geo are both zero
#                 if x_geo == 0 and y_geo == 0:
#                     glon = 0.0  # Longitude is undefined, set to 0 or handle as needed
#                 else:
#                     glon = np.rad2deg(np.arctan2(y_geo, x_geo))
#                 glat = np.rad2deg(np.arctan2(z_geo, np.sqrt(x_geo**2 + y_geo**2)))

#                 result = (glat, glon, 0)  # Use 0 for height, similar to other methods
#             glats.append(result[0])
#             glons.append(result[1])

#         mlats_dct[mlat] = {'glats': glats, 'glons': glons}

#         if is_saved: # 'is_saved is True' can be simplified to 'is_saved'
#             logger.info('Saving magnetic graticule for %f degrees magnetic latitude.', mlat)
#             # Use f-strings for cleaner string formatting
#             filename = f'Graticule_{mlat}_{ut.strftime("%Y%m%d%H%M%S")}_{method}'
#             directory = 'output' # Define base directory once
#             output_dir_gpx = os.path.join(directory, 'graticules')
#             output_dir_geojson = os.path.join(directory, 'geojson')

#             # Ensure output directories exist
#             os.makedirs(output_dir_gpx, exist_ok=True)
#             os.makedirs(output_dir_geojson, exist_ok=True)

#             df = pd.DataFrame({'GLAT': mlats_dct[mlat]['glats'],
#                                'GLON': mlats_dct[mlat]['glons']})

#             # --- GPX Saving ---
#             gpx = gpxpy.gpx.GPX()
#             gpx_track = gpxpy.gpx.GPXTrack()
#             gpx.tracks.append(gpx_track)
#             gpx_segment = gpxpy.gpx.GPXTrackSegment()
#             gpx_track.segments.append(gpx_segment)

#             for idx in df.index:
#                 gpx_segment.points.append(gpxpy.gpx.GPXTrackPoint(
#                     latitude=df.loc[idx, 'GLAT'], longitude=df.loc[idx, 'GLON']))

#             # It's better to log the XML content after writing to file, or if debugging
#             # logger.info(gpx.to_xml())

#             gpx_filepath = os.path.join(output_dir_gpx, filename + '.gpx')
#             with open(gpx_filepath, 'w', encoding="utf-8") as f:
#                 f.write(gpx.to_xml())
#                 logger.info("Writing %s to gpx.", gpx_filepath)

#             # --- GeoJSON Saving ---
#             features = []
#             # For a LineString, you'd collect all coordinates into one list
#             coordinates = []
#             for idx in df.index:
#                 # GeoJSON coordinates are [longitude, latitude]
#                 coordinates.append([df.loc[idx, 'GLON'], df.loc[idx, 'GLAT']])

#             # Create a single LineString feature for the ring
#             line_feature = {
#                 "type": "Feature",
#                 "geometry": {
#                     "type": "LineString",
#                     "coordinates": coordinates
#                 },
#                 "properties": {
#                     "magnetic_latitude": mlat,
#                     "datetime_utc": ut.isoformat(),
#                     "method": method
#                 }
#             }
#             features.append(line_feature)


#             # Create a GeoJSON FeatureCollection
#             geojson_output = {
#                 "type": "FeatureCollection",
#                 "features": features
#             }

#             geojson_filepath = os.path.join(output_dir_geojson, filename + '.geojson')
#             with open(geojson_filepath, 'w', encoding='utf-8') as f:
#                 json.dump(geojson_output, f, ensure_ascii=False, indent=4)
#                 logger.info("Writing %s to geojson.", geojson_filepath)

#     return mlats_dct


# rings = calc_mlat_rings(list(range(-90, 90, 5)), ut = dt.datetime(2020, 1, 1), is_saved = True)

In [14]:
# rings

NameError: name 'rings' is not defined

In [ ]:
    # import togeojson
    # from xml.dom import minidom

    # def gpx_to_geojson(gpx_file_path, output_file_path):
    #     """Converts a GPX file to GeoJSON format."""
    #     try:
    #         with open(gpx_file_path, 'r') as f:
    #             gpx_content = f.read()

    #         gpx_dom = minidom.parseString(gpx_content)
    #         geojson_data = togeojson.gpx(gpx_dom)

    #         with open(output_file_path, 'w') as outfile:
    #             import json
    #             json.dump(geojson_data, outfile, indent=2)

    #         print(f"Successfully converted {gpx_file_path} to {output_file_path}")

    #     except Exception as e:
    #         print(f"Error during conversion: {e}")

    # # Example usage:
    # gpx_file = 'output/graticules/Graticule_ -10_2020-01-01 00:00:00_auto.gpx' # Replace with your GPX file path
    # geojson_file = 'output/graticules/Graticule_ -10_2020-01-01 00:00:00_auto.geojson' # Replace with your desired output file path
    # gpx_to_geojson(gpx_file, geojson_file)

In [ ]:
# from geojson_transformer import GeoJsonTransformer

# # Create an object from a gpx file.
# gpxfile = GeoJsonTransformer(path='output/graticules/Graticule_ -10_2020-01-01 00:00:00_auto.gpx') 

# # Returns the total distance between each point in the gpx file.
# gpxfile.total_distance

# # Returns the total cumulative elevation between each point in the gpx file.
# gpxfile.total_elevation

# # Returns the first lat/lon pair found in the file.
# gpxfile.starting_point

In [ ]:
gpxfile.save_geojson(filepath="output/graticules/foo.geojson")

In [24]:
for method in ["gsm", "qdip", "apex", "aacgm"]:
    rings = conj.calc_mlat_rings([-85], ut =
                        dt.datetime(2020, 1, 1), method = method, is_saved = True)
    

# 56.49422103984533
# -0.6557771833120372, -73.69442757234994


In [25]:
pd.DataFrame(rings)

,-85
glats,"[-78.99419713663143, -79.02959400506468, -79.0..."
glons,"[134.41388878371671, 134.0054413920879, 133.59..."


In [ ]:
type(rings)

In [ ]:
help(conj.calc_mlat_rings)